In [5]:
import cv2
import os

total_no_of_images = len(os.listdir('images'))
count = 0
for item, file in enumerate(os.listdir('images')):
    src = cv2.imread(cv2.samples.findFile('images/'+file)) if 'raw' in file else None
    if src is None:
        continue
    blurred = cv2.GaussianBlur(src, (11, 11), 0)
    width, height = src.shape[:2]
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, (10, 100, 100), (20, 255, 255))
    mask = cv2.erode(mask, None, iterations=2)
    mask = cv2.dilate(mask, None, iterations=2)
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)
    new_cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    if len(new_cnts) > 0:
        c = max(new_cnts, key=cv2.contourArea)
        # for c in new_cnts:
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        M = cv2.moments(c)
        center = int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]),
        # To see the centroid clearly\n",
        if radius > 10 and radius < 65:
            cv2.circle(src, center, int(radius), (0, 255, 0), 2)
            cv2.circle(src, center, 1, (0, 0, 255), 2)
            cv2.imwrite(f"contour_detection_results/{file}", src)
    cv2.imshow('result', src)
    cv2.waitKey(1000)
    cv2.destroyAllWindows()
    if item == 10:
        break
# print(count/(total_no_of_images//2))